# Instituto Federal do Sudeste de Minas Gerais, Campus Barbacena

# Minicurso: Introdução a word embeddings



## 1. Word2vec - Mais exemplos


### 1.1. Game of Thrones

Alguns resultados de um treinamento onde utilizzei como dataset os 3 primeiros livros de Game of Thrones.

#### 1.1.1. Similaridade

A similaridade entre robb e stark é muito maior do que a similaridade entre tyrion e  stark, o que faz sentido visto que Robb é da família Stark enquanto Tyrion é um Lannister.
![](img/got1.png)

Exemplo de similaridade com palavras mais comuns.
![](img/got3.png)

As 10 palavras mais similares relacionadas aos sobrenomes Targaryen e Lannister.
![](img/got4.png)

### 1.1.2. Odd-One-Out

Odd-One-Out aplicado a personagens dos Livros. É interessante como o modelo consegue isolar personagens pertencentes a famílias diferentes.

![](img/got2.png)

### 1.2. Os Simpsons

Mais exemplos de resultados com o treinamento do wor2vec, dessa vez aplicado a um dataset com dados do desenho "Os Simpsons".

Fonte: https://www.kaggle.com/pierremegret/gensim-word2vec-tutorial

### 1.2.1. Similaridade

As 10 palavras mais similares a homer.
![](img/smp1.png)

As 10 palavras mais similares à bart.
![](img/smp2.png)

### 1.2.2. Odd-One-Out

Mais exemplos de Odd-One-Out isolando com sucesso personagens fora do contexto.
![](img/smp3.png)
![](img/smp4.png)
![](img/smp5.png)

### 1.2.3. Analogias

woman está para marge como homer está para...
![](img/smp6.png)
woman está para man como bart está para...
![](img/smp8.png)

### 1.3. Wikipedia em Português

Utilizando um modelo treinado em Português com os dados da wikipédia.
Esse modelo foi treinado por um coreano, que utilizou a base da wikipedia em portugues. Essa base contem cerca de 1,3 GB e resultdou em um vocabulário de aproximadamente 50.000 palavras. A dimensionalidade escolhida por ele foi a mesma usada pelo modelo do google, 300.



Vamos baixar esse modelo e fazer alguns testes, ele pode ser encontrado no seguinte link:  https://github.com/KyuByong/wordvectors

In [39]:
#Carregando o modelo
model_wiki = Word2Vec.load('./pt/pt.bin')

In [48]:
#vocabulario de 50246 palavras.
len(model_wiki.wv.vocab)

50246

### 1.3.1. Similadirade entre palavras

In [43]:
model_wiki.wv.similarity('rei','rainha')

0.5698206

In [44]:
model_wiki.wv.similarity('rei','principe')

0.5398298

In [45]:
model_wiki.wv.similarity('rei','cachorro')

0.17778099

### 1.3.2. Os mais similares

In [50]:
model_wiki.wv.most_similar(['rei'])

[('monarca', 0.7494288086891174),
 ('imperador', 0.7331966161727905),
 ('soberano', 0.7012052536010742),
 ('sultão', 0.6885693073272705),
 ('príncipe', 0.6859150528907776),
 ('faraó', 0.6376820206642151),
 ('governante', 0.6339693665504456),
 ('califa', 0.6284059286117554),
 ('trono', 0.62713623046875),
 ('condestável', 0.626724362373352)]

In [51]:
model_wiki.wv.most_similar(['rainha'])

[('princesa', 0.7254871129989624),
 ('rainha-mãe', 0.715790867805481),
 ('imperatriz', 0.6730843782424927),
 ('infanta', 0.6339465379714966),
 ('czarina', 0.613501787185669),
 ('grã-duquesa', 0.5965134501457214),
 ('duquesa', 0.5820716619491577),
 ('sibila', 0.5797038674354553),
 ('arquiduquesa', 0.571153461933136),
 ('rei', 0.5698206424713135)]

In [53]:
model_wiki.wv.most_similar(['cachorro'])

[('gato', 0.7345885038375854),
 ('cão', 0.7251510620117188),
 ('macaco', 0.712258517742157),
 ('hamster', 0.6716378927230835),
 ('burro', 0.6668468713760376),
 ('gorila', 0.6535876989364624),
 ('rato', 0.6522855758666992),
 ('sapo', 0.6512554883956909),
 ('urso', 0.6504798531532288),
 ('pássaro', 0.649040937423706)]

In [54]:
model_wiki.wv.most_similar(['conhecimento'])

[('entendimento', 0.6994889974594116),
 ('pensamento', 0.6006251573562622),
 ('discernimento', 0.5953051447868347),
 ('aprendizado', 0.5870758295059204),
 ('conhecimentos', 0.5852985382080078),
 ('interesse', 0.5579639673233032),
 ('fundamento', 0.5576693415641785),
 ('significado', 0.5541521310806274),
 ('intelecto', 0.5460195541381836),
 ('desconhecimento', 0.5352515578269958)]

In [55]:
model_wiki.wv.most_similar(['amor'])

[('ódio', 0.6318349242210388),
 ('encanto', 0.6172047853469849),
 ('ciúme', 0.5946954488754272),
 ('afeto', 0.5838000774383545),
 ('beijo', 0.5753592252731323),
 ('ternura', 0.5546466708183289),
 ('prazer', 0.553915798664093),
 ('arrependimento', 0.5529347658157349),
 ('paixão', 0.5403603315353394),
 ('compaixão', 0.5381144285202026)]

### 1.3.3. Analogias

In [75]:
model_wiki.wv.most_similar(positive=['principe', 'mulher'], negative=['homem'], topn=1)

[('princesa', 0.5795178413391113)]

In [76]:
model_wiki.wv.most_similar(positive=['nuvem', 'estrelas'], negative=['nuvens'], topn=1)

[('estrela', 0.5891047120094299)]

### 1.3.2. Odd-One-Out

In [67]:
model_wiki.wv.doesnt_match(['principe', 'princesa', 'monarca', 'alface'])

'alface'

In [77]:
model_wiki.wv.doesnt_match(['futebol', 'volei', 'natacao', 'gato'])

'gato'

## 2. Treinando seu Modelo

Hora de por em prática o que foi visto e treinar seu pŕoprio modelo.


### 2.1. Criando o Corpus

In [5]:
import numpy as np
import pandas as pd
from unidecode import unidecode
import nltk
import os
import re
from gensim.models import Word2Vec, KeyedVectors
from string import punctuation
#Pode ser alterado de acordo com a banda ou parcela do dataset escolhida. Existem musicas em diferentes linguas no dataset.
stopwords = nltk.corpus.stopwords.words('english')
#stopwords = nltk.corpus.stopwords.words('portuguese')
from nltk.stem import SnowballStemmer

In [6]:
pontuacao = list(punctuation)
# Função pra remover pontuação, stopwords e numeros de textos, pois essas são informações irrelevantes 
pontuacao.append('...')
pontuacao.append('\"')
pontuacao.append('\'')
pontuacao.append('``')
pontuacao.append('`')
pontuacao.append('\n')
pontuacao.append('\t')
pontuacao.append('\’')

def processa(txt):
    for p in pontuacao:
        txt = str(txt).replace(p, ' ')
    return ' '.join([t for t in txt.lower().split() 
                     if (len(t)>2) 
                     if not re.search('^\d', t) 
                     and not re.search('.*\d$', t) 
                     and (t not in stopwords)])

In [7]:
# Lendo os datasets
df1 = pd.read_csv('./music/lyrics1.csv')
df2 = pd.read_csv('./music/lyrics2.csv')
df3 = pd.read_csv('./music/lyrics3.csv')
df4 = pd.read_csv('./music/lyrics4.csv')
df5 = pd.read_csv('./music/lyrics5.csv')
df6 = pd.read_csv('./music/lyrics6.csv')
df7 = pd.read_csv('./music/lyrics7.csv')
df8 = pd.read_csv('./music/lyrics8.csv')
df9 = pd.read_csv('./music/lyrics9.csv')

In [8]:
# Concatenando os 2 datasets
df =  pd.concat([df1, df2, df3, df4, df5, df6, df7, df8, df9])
df.head()

,Unnamed: 0,Band,Lyrics,Song
0,0,Elijah Blake,"No, no\r\nI ain't ever trapped out the bando\r...",Everyday
1,1,Elijah Blake,"The drinks go down and smoke goes up, I feel m...",Live Till We Die
2,2,Elijah Blake,She don't live on planet Earth no more\r\nShe ...,The Otherside
3,3,Elijah Blake,"Trippin' off that Grigio, mobbin', lights low\...",Pinot
4,4,Elijah Blake,"I see a midnight panther, so gallant and so br...",Shadows & Diamonds


In [9]:
# 516174 musicas disponíveis no dataset
df.shape

(516174, 4)

In [10]:
# 42627 Cantores/Bandas diferentes no dataset
len(df['Band'].unique())

42627

In [11]:
# Escolha uma ou mais bandas para treinar seu modelo
minhas_musicas = df[df['Band']=='Imagine Dragons']

In [12]:
# As 5 primeiras musicas da parcela escolhida
minhas_musicas.head()

,Unnamed: 0,Band,Lyrics,Song
41280,211280,Imagine Dragons,Ever since I could remember\r\nEverything insi...,Monster
41281,211281,Imagine Dragons,When the days are cold\nAnd the cards all fold...,Demons
41282,211282,Imagine Dragons,So this is what you meant\r\nWhen you said tha...,It's Time
41283,211283,Imagine Dragons,"I took a walk on a Saturday night,\nFog in the...",My Fault
41284,211284,Imagine Dragons,If you love somebody\r\nBetter tell them why t...,On Top of the World


In [13]:
# 60 musicad da banda escolhida. Escolha uma bonda com um numero consideravel de musicas
minhas_musicas.shape

(60, 4)

In [14]:
# Criando o corpus. Vale ressaltar que o corpus é uma lista de listas.
corpus = []

In [15]:
# Percorre linha or linha do dataset
for i in minhas_musicas.iterrows():
    corpus.append(processa(unidecode(str(i[1]['Lyrics']))).split(' '))

In [16]:
# Conferindo o corpus
corpus;

### 2.2. Treinando o Modelo

Experimente parâmetros diferentes, outras combinação de técnicas de pré-processamento, tente aprimorar os resultados. A dimensionalidade e o tamanho da janela são os que vão causa uma mudança mais expressiva. O motivo disso é testar, para perceber como os datasets reagem a cada configuração, e chegar a sua pŕopria conclusão de qual combinação de parâmetros e técnicas de pré-proessamento se adequa melhor à parela do daset escolhida, ou até mesmo a todo o dataset.


Outro experimento interessante é usar a mesma configuração pra treinar modelos com bandas diferentes e ver como palavras iguais são definidas em contextos diferentes para cada banda.

In [17]:
# Treinando o modelo 
from gensim.models import Word2Vec
model = Word2Vec(corpus, size=300, window=5, min_count=3, sg=0, iter=50)

In [18]:
model.train(corpus, total_examples=len(corpus), epochs=50)

(240699, 353800)

In [19]:
model.wv.most_similar(['love', 'lost'], topn=10)

[('real', 0.7288336753845215),
 ('dangerous', 0.7069138884544373),
 ('way', 0.665067195892334),
 ('trust', 0.6614406108856201),
 ('guess', 0.657089352607727),
 ('feels', 0.6397360563278198),
 ('rings', 0.595757007598877),
 ('something', 0.5861997604370117),
 ('headlights', 0.5847240686416626),
 ('try', 0.5481896996498108)]

In [20]:
model.wv.most_similar(['adrenaline'], topn=10)

[('takes', 0.9612326622009277),
 ('whatever', 0.9375436305999756),
 ('chains', 0.8245683908462524),
 ('veins', 0.7950506210327148),
 ('love', 0.7021576166152954),
 ('leaves', 0.6828089952468872),
 ('rip', 0.675251841545105),
 ('feels', 0.6737320423126221),
 ('cause', 0.6270672678947449),
 ('break', 0.6024526953697205)]

### 2.3 Words Movers Distance

Metodo utilizado para comparar setenças ou documentos.

In [78]:
import warnings
warnings.filterwarnings('ignore')
model.init_sims(replace=True) 
model.wmdistance(corpus[0], corpus[1])

0.9220106665984066

In [79]:
model.wmdistance(corpus[0], corpus[2])

1.0239031157176919

In [80]:
model.wmdistance(corpus[0], corpus[3])

1.0590478900508051